In [3]:
from hydra import compose, initialize
import torch
import torch.nn as nn
import torch.nn.functional as F
import mlflow
import optuna
import numpy as np
from tqdm import tqdm
from typing import Tuple
from omegaconf.omegaconf import OmegaConf
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR

from model import UCCDRNModel
from dataset import MnistDataset
from utils import get_or_create_experiment, parse_experiment_runs_to_optuna_study
torch.autograd.set_detect_anomaly(True)


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True


def init_model_and_optimizer(args, model_cfg, device):
    model = UCCDRNModel(model_cfg).to(device)
    # optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    param_group_names = [name for name, _ in model.named_parameters()]
    optimizer = torch.optim.Adam(
        model.parameters(), lr=args.learning_rate, amsgrad=True)
    # for name, param_group in zip(param_group_names, optimizer.param_groups):
    #     if "encoder" in name:
    #         param_group['lr'] = args.learning_rate*args.lr_multiplier
    #     print(f'    {name}: {param_group["lr"]}')
    return model, optimizer


def init_dataloader(args):
    # assert args.dataset in [
    #     "mnist",
    #     "camelyon",
    # ], "Mode should be either mnist or camelyon"
    # if args.dataset == "mnist":
    train_dataset_len = args.train_num_steps * args.batch_size
    train_dataset = MnistDataset(
        mode="train",
        num_instances=args.num_instances,
        num_samples_per_class=args.num_samples_per_class,
        digit_arr=list(range(args.ucc_end-args.ucc_start+1)),
        ucc_start=args.ucc_start,
        ucc_end=args.ucc_end,
        length=train_dataset_len,
    )
    val_dataset_len = args.val_num_steps * args.batch_size
    val_dataset = MnistDataset(
        mode="val",
        num_instances=args.num_instances,
        num_samples_per_class=args.num_samples_per_class,
        digit_arr=list(range(args.ucc_end-args.ucc_start+1)),
        ucc_start=args.ucc_start,
        ucc_end=args.ucc_end,
        length=val_dataset_len,
    )
    # create dataloader
    train_loader = DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        shuffle=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        shuffle=False,
    )
    return train_loader, val_loader


def evaluate(model, val_loader, device, ae_mode, clf_mode) -> dict:
    model.eval()
    val_ae_loss_list = []
    val_ucc_loss_list = []
    val_acc_list = []
    rec_criterion = nn.MSELoss()
    with torch.no_grad():
        for batch_samples, batch_labels in val_loader:
            batch_samples = batch_samples.to(device)
            batch_labels = batch_labels.to(device)
            if ae_mode:
                batch_size, num_instances, num_channel, patch_size, _ = batch_samples.shape
                x = batch_samples.view(-1, num_channel,
                                       batch_samples.shape[-2], batch_samples.shape[-1])
                features = model.encoder(x)
                reconstruction = model.decoder(features)
                reconstruction = reconstruction.view(batch_size, num_instances,
                                    1, patch_size, patch_size)
                ae_loss = rec_criterion(batch_samples, reconstruction)
                val_ae_loss_list.append(ae_loss.item())

            if clf_mode:
                ucc_logits, _ = model(batch_samples)
                ucc_val_loss = F.cross_entropy(ucc_logits, batch_labels)
                # acculate accuracy
                _, ucc_predicts = torch.max(ucc_logits, dim=1)
                acc = torch.sum(
                    ucc_predicts == batch_labels).item() / len(batch_labels)
                val_acc_list.append(acc)
                val_ucc_loss_list.append(ucc_val_loss.item())

        if ae_mode and clf_mode:
            return {
                "eval_ae_loss": np.round(np.mean(val_ae_loss_list), 5),
                "eval_ucc_loss": np.round(np.mean(val_ucc_loss_list), 5),
                "eval_ucc_acc": np.round(np.mean(val_acc_list), 5)
            }
        elif ae_mode:
            return {
                "eval_ae_loss": np.round(np.mean(val_ae_loss_list), 5),
            }
        elif clf_mode:
            return {
                "eval_ucc_loss": np.round(np.mean(val_ucc_loss_list), 5),
                "eval_ucc_acc": np.round(np.mean(val_acc_list), 5)
            }


def train(args, model, optimizer, lr_scheduler, train_loader, val_loader, device):
    print("training")
    # mlflow.pytorch.log_model(model, "init_model")
    # output_dir = hydra.core.hydra_config.HydraConfig.get().runtime.output_dir
    model.train()
    step = 0
    best_eval_acc = 0
    patience = 2
    ae_steps = 500

    rec_criterion = nn.MSELoss()
    if step == 0:
        mlflow.pytorch.log_model(
            model,
            "best_model.pth"
        )
    for batch_samples, batch_labels in tqdm(train_loader):
        batch_samples = batch_samples.to(device)
        batch_labels = batch_labels.to(device)
        optimizer.zero_grad()

        if ae_steps > 0:
            batch_size, num_instances, num_channel, patch_size, _ = batch_samples.shape
            x = batch_samples.view(-1, num_channel,
                                   batch_samples.shape[-2], batch_samples.shape[-1])
            feature = model.encoder(x)

            reconstruction = model.decoder(feature)
            reconstruction = reconstruction.view(
                batch_size, num_instances, 1, patch_size, patch_size)
            loss = rec_criterion(batch_samples, reconstruction)
            ae_loss = loss
            ae_steps -= 1

        if ae_steps == 0:
            ucc_logits, reconstruction = model(batch_samples, batch_labels)
            loss: torch.Tensor = F.cross_entropy(
                ucc_logits,
                batch_labels
            )
            ae_loss = F.mse_loss(batch_samples, reconstruction)

        loss.backward()

        optimizer.step()

        step += 1

        if step % 10 == 0:
            with torch.no_grad():
                metric_dict = {}
                grad_log = {name: torch.mean(param.grad).cpu().item(
                ) for name, param in model.named_parameters() if isinstance(param.grad, torch.Tensor)}
                mlflow.log_metrics(grad_log, step=step)
                metric_dict["train_ae_loss"] = np.round(
                    ae_loss.detach().item(), 5)
                if ae_steps == 0:
                    _, pred = torch.max(ucc_logits, dim=1)
                    accuracy = torch.sum(
                        pred.flatten() == batch_labels.flatten())/len(batch_labels)
                    metric_dict["train_ucc_loss"] = np.round(
                        loss.detach().item(), 5)
                    metric_dict["train_ucc_acc"] = np.round(float(accuracy), 5)

            mlflow.log_metrics(metric_dict, step=step)

        if step % args.save_interval == 0:

            eval_metric_dict = evaluate(
                model,
                val_loader,
                device,
                ae_mode=True,
                clf_mode=(ae_steps == 0))

            print(
                f"step: {step}," + ",".join([f"{key}: {value}"for key, value in eval_metric_dict.items()]))
            mlflow.log_metrics(eval_metric_dict, step=step)
            # early stop
            if ae_steps == 0:
                eval_acc = eval_metric_dict["eval_ucc_acc"]
                if eval_acc > best_eval_acc:
                    patience = 2
                    best_eval_acc = eval_acc
                    mlflow.pytorch.log_model(
                        model,
                        "best_model.pth"
                    )
                else:
                    patience -= 1

            if patience <= 0:
                break
            if step == 10000:
                break
            model.train()

    print("Training finished!!!")
    return best_eval_acc

In [4]:
def objective(trial: optuna.Trial, cfg_name):
    print(cfg_name)
    with mlflow.start_run(nested=True):
        # cfg = OmegaConf.load("../configs/train_drn.yaml")
        with initialize(version_base=None, config_path="../configs"):
            cfg = compose(config_name=cfg_name)
        # with open("params.json", "r") as file:
        #     params_config = json.loads(file.read())

        defaults = {
            # "init_method": {
            #     "type": "categorical",
            #     "range": ["uniform", "normal", "xavier_uniform", "xavier_normal", "kaiming_uniform", "kaiming_normal"],
            #     "aliases": [
            #         "model.drn.init_method",
            #     ]
            # },
            # "num_bins": {
            #     "type": "int",
            #     "value": 10,
            #     "range": [5, 100],
            #     "aliases": [
            #         "model.drn.num_bins",
            #         "args.num_bins",
            #         "model.kde_model.num_bins"
            #     ]
            # },
            "lr": {
                "type": "float",
                "value": 0.005,
                "range": [0.008, 0.08],
                "aliases": ["args.learning_rate"]
            },
            "hidden_q": {
                "type": "int",
                "value": 100,
                "range": [4, 100],
                "aliases": ["model.drn.hidden_q"]
            },
            "num_layers": {
                "type": "int",
                "value": 2,
                "range": [1, 10],
                "aliases": ["model.drn.num_layers"]
            },
            "num_nodes": {
                "type": "int",
                "value": 9,
                "range": [1, 10],
                "aliases": ["model.drn.num_nodes"]
            }
        }
        for key, value in defaults.items():
            if "value" in value:
                v = value["value"]
            else:
                if value["type"] == "int":
                    v = trial.suggest_int(
                        key, value["range"][0], value["range"][1])
                elif value["type"] == "categorical":
                    v = trial.suggest_categorical(key, value["range"])
                else:
                    v = trial.suggest_float(
                        key, value["range"][0], value["range"][1])
            for a in value["aliases"]:
                exec(f"cfg.{a} = '{v}'") if isinstance(
                    v, str) else exec(f"cfg.{a} = {v}")

        print(cfg)
        mlflow.log_dict(dict(OmegaConf.to_object(cfg)), "config.yaml")

        params = trial.params
        for key, value in params.items():
            mlflow.log_param(key=key, value=value)

        args = cfg.args
        device = torch.device("cuda" if torch.cuda.is_available() else "mps")
        model, optimizer = init_model_and_optimizer(args, cfg, device)
        train_loader, val_loader = init_dataloader(args)
        mlflow.pytorch.log_model(model, "init_model")
        best_acc = train(args, model, optimizer, None,
                         train_loader, val_loader, device)
    return best_acc


print("run this once")
mlflow.set_tracking_uri("mlruns")
run_name = "ucc-drn-multi-step"
experiment_id = get_or_create_experiment(experiment_name=run_name)
mlflow.set_experiment(experiment_id=experiment_id)
cfg_name = "train_drn_multi_step"
study = parse_experiment_runs_to_optuna_study(
    experiment_name=run_name,
    study_name=run_name,
    cfg_name=cfg_name,
    params_file="params-init.json",
    experiments=False
)
study.optimize(lambda trial: objective(trial, cfg_name), n_trials=100, show_progress_bar=True)

run this once


[I 2025-05-20 21:47:50,332] A new study created in memory with name: ucc-drn-multi-step
  0%|          | 0/100 [00:00<?, ?it/s]

train_drn_multi_step
{'args': {'dataset': 'mnist', 'model_dir': 'saved_models/', 'model_name': 'mnist_ucc_drn', 'num_instances': 32, 'ucc_start': 1, 'ucc_end': 4, 'batch_size': 20, 'num_samples_per_class': 5, 'num_workers': 4, 'learning_rate': 0.005, 'num_bins': 11, 'num_features': 10, 'train_num_steps': 100000, 'val_num_steps': 200, 'save_interval': 1000, 'seed': 22}, 'model': {'num_channels': 1, 'input_shape': [28, 28, 1], 'kde_model': {'num_bins': 11, 'sigma': 0.1}, 'encoder': {'conv_input_channel': 1, 'conv_output_channel': 16, 'block1_output_channel': 321, 'block1_num_layer': 1, 'block2_output_channel': 64, 'block2_num_layer': 1, 'block3_output_channel': 128, 'block3_num_layer': 1, 'flatten_size': 6272, 'num_features': 10}, 'decoder': {'linear_size': 6272, 'reshape_size': [7, 7, 128], 'block1_output_channel': 64, 'block1_num_layer': 1, 'block2_output_channel': 32, 'block2_num_layer': 1, 'block3_output_channel': 16, 'block3_num_layer': 1, 'output_channel': 1}, 'drn': {'init_method'

2025/05/20 21:49:01 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/20 21:49:24 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/20 21:49:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 21:49:24 WARNING mlflow.utils.requirement

training


...

2025/05/23 18:54:17 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/23 18:54:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


step: 2000,eval_ae_loss: 0.69242,eval_ucc_loss: 1.3863,eval_ucc_acc: 0.25


step: 3000,eval_ae_loss: 0.69174,eval_ucc_loss: 1.38646,eval_ucc_acc: 0.25


Best trial: 84. Best value: 0.997: 100%|██████████| 100/100 [69:24:42<00:00, 2498.83s/it]

Training finished!!!
[I 2025-05-23 19:13:36,264] Trial 175 finished with value: 0.25 and parameters: {}. Best is trial 84 with value: 0.997.


In [28]:
experiment_id


'823019999500194436'

In [27]:
import pandas as pd

with pd.option_context("display.max_columns", None):
    display(runs.sort_values(by="metrics.train_ucc_acc", ascending=False))

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.encoder.2.blocks.0.conv2.bias,metrics.encoder.2.blocks.0.conv1.weight,metrics.ucc_classifier.1.lama,metrics.encoder.1.blocks.0.conv1.bias,metrics.ucc_classifier.1.ba,metrics.encoder.6.weight,metrics.decoder.3.blocks.0.conv1.bias,metrics.encoder.0.weight,metrics.decoder.0.weight,metrics.encoder.1.blocks.0.conv1.weight,metrics.eval_ae_loss,metrics.eval_ucc_acc,metrics.ucc_classifier.0.W,metrics.ucc_classifier.2.lamq,metrics.encoder.2.blocks.0.skip_conv.bias,metrics.ucc_classifier.0.lamq,metrics.train_ucc_acc,metrics.ucc_classifier.0.ba,metrics.encoder.2.blocks.0.conv1.bias,metrics.decoder.3.blocks.0.conv1.weight,metrics.ucc_classifier.2.W,metrics.ucc_classifier.1.lamq,metrics.decoder.5.blocks.0.skip_conv.bias,metrics.encoder.3.blocks.0.skip_conv.bias,metrics.encoder.1.blocks.0.skip_conv.bias,metrics.decoder.7.weight,metrics.decoder.4.blocks.0.skip_conv.bias,metrics.decoder.5.blocks.0.conv1.bias,metrics.encoder.0.bias,metrics.ucc_classifier.1.W,metrics.decoder.3.blocks.0.conv2.bias,metrics.decoder.5.blocks.0.conv2.weight,metrics.encoder.3.blocks.0.skip_conv.weight,metrics.encoder.3.blocks.0.conv2.weight,metrics.encoder.2.blocks.0.skip_conv.weight,metrics.encoder.1.blocks.0.skip_conv.weight,metrics.encoder.1.blocks.0.conv2.bias,metrics.encoder.1.blocks.0.conv2.weight,metrics.ucc_classifier.0.bq,metrics.train_ucc_loss,metrics.encoder.3.blocks.0.conv1.bias,metrics.ucc_classifier.1.bq,metrics.encoder.3.blocks.0.conv2.bias,metrics.decoder.5.blocks.0.skip_conv.weight,metrics.decoder.4.blocks.0.conv1.bias,metrics.decoder.7.bias,metrics.decoder.0.bias,metrics.decoder.4.blocks.0.skip_conv.weight,metrics.ucc_classifier.0.lama,metrics.ucc_classifier.2.ba,metrics.decoder.5.blocks.0.conv2.bias,metrics.decoder.3.blocks.0.conv2.weight,metrics.train_ae_loss,metrics.decoder.3.blocks.0.skip_conv.weight,metrics.encoder.3.blocks.0.conv1.weight,metrics.decoder.3.blocks.0.skip_conv.bias,metrics.decoder.4.blocks.0.conv1.weight,metrics.decoder.4.blocks.0.conv2.weight,metrics.eval_ucc_loss,metrics.ucc_classifier.2.lama,metrics.encoder.2.blocks.0.conv2.weight,metrics.decoder.4.blocks.0.conv2.bias,metrics.decoder.5.blocks.0.conv1.weight,metrics.ucc_classifier.2.bq,metrics.best_eval_ucc_acc,params.init_method,params.hidden_q,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.log-model.history,tags.mlflow.source.type
1,e7bd69f493604ce99a968daae16c9822,823019999500194436,FINISHED,file:///d:/UCC-DRN-Pytorch/mnist/mlruns/823019...,2025-05-21 08:25:14.419000+00:00,2025-05-21 13:44:19.916000+00:00,-7.013838e-07,-2.449375e-07,1.394311e-03,1.224577e-05,-0.000008,5.686255e-06,0.000203,-0.000103,0.000011,2.529096e-06,1.33154,0.99275,0.000193,0.005338,-8.903549e-05,0.002112,1.00,-0.000720,-4.887571e-06,0.000002,-0.000643,0.000507,0.001259,0.000037,0.000039,0.043668,0.003322,-0.000868,0.000022,0.000194,0.000027,-0.000295,-5.159472e-07,-2.011760e-07,-2.643304e-06,-0.000016,9.073563e-07,-1.349980e-06,-0.000338,0.77835,1.808582e-06,0.000158,5.896663e-08,0.000646,0.000218,0.065277,2.234133e-05,0.001868,1.843372e-03,-0.001664,-0.000880,0.000002,1.32782,0.000191,2.049135e-07,0.001598,-0.000318,-4.555302e-05,0.79035,8.910578e-02,4.919872e-07,-0.000164,-0.000366,-0.003957,NaN,None,None,d:\UCC-DRN-Pytorch\.venv\lib\site-packages\ipy...,melodic-conch-890,guanyu,"[{""run_id"": ""e7bd69f493604ce99a968daae16c9822""...",LOCAL
3,76d45e8fdc07489fbd155c3ac5142008,823019999500194436,FINISHED,file:///d:/UCC-DRN-Pytorch/mnist/mlruns/823019...,2025-05-17 18:55:38.015000+00:00,2025-05-20 13:48:43.612000+00:00,-1.268455e-07,-5.323526e-07,-1.717934e-11,2.432823e-06,-0.000205,2.139346e-05,0.000115,-0.000895,0.000001,-3.548650e-06,1.22984,0.90875,-0.000611,0.110755,-1.985141e-04,-0.007071,0.90,-0.001036,-1.484229e-06,0.000012,0.002763,0.003129,-0.001913,0.000018,-0.000233,0.018503,-0.000811,0.000200,-0.000153,-0.000070,0.000149,-0.000036,-5.043293e-06,-8.023744e-07,-3.371045e-05,-0.000230,3.357969e-07,8.

In [ ]:
e7bd69f493604ce99a968daae16c9822

Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.encoder.2.blocks.0.conv2.bias',
       'metrics.encoder.2.blocks.0.conv1.weight',
       'metrics.ucc_classifier.1.lama',
       'metrics.encoder.1.blocks.0.conv1.bias', 'metrics.ucc_classifier.1.ba',
       'metrics.encoder.6.weight', 'metrics.decoder.3.blocks.0.conv1.bias',
       'metrics.encoder.0.weight', 'metrics.decoder.0.weight',
       'metrics.encoder.1.blocks.0.conv1.weight', 'metrics.eval_ae_loss',
       'metrics.eval_ucc_acc', 'metrics.ucc_classifier.0.W',
       'metrics.ucc_classifier.2.lamq',
       'metrics.encoder.2.blocks.0.skip_conv.bias',
       'metrics.ucc_classifier.0.lamq', 'metrics.train_ucc_acc',
       'metrics.ucc_classifier.0.ba', 'metrics.encoder.2.blocks.0.conv1.bias',
       'metrics.decoder.3.blocks.0.conv1.weight', 'metrics.ucc_classifier.2.W',
       'metrics.ucc_classifier.1.lamq',
       'metrics.decoder.5.blocks.0.skip_conv.bias',
       'metrics